In [1]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re, time

pd.options.display.max_colwidth = 1000

In [2]:
from loadFile import getPathList, fileToDataFrame

# ディレクトリ 内のメールファイルを読み込む
directory_path = "enron/maildir/allen-p/inbox"
file_list, dir_list=getPathList(directory_path)

# テキストファイルをデータフレームに格納する
df = fileToDataFrame(file_list)

In [3]:
#ドキュメント（メッセージ）をセンテンスに分解する
from preProcess import toSentences

sent_dict = {'Message-ID':[],
            'Sentence-ID':[],
            'Sentence':[]}

start = time.time()

for i,message in enumerate(df['Message']):
    sentences = toSentences(message)
    sent_dict['Message-ID'].extend([df['Message-ID'][i]] * len(sentences))
    sent_dict['Sentence-ID'].extend(range(len(sentences)))
    sent_dict['Sentence'].extend(sentences)

sentence_df = pd.DataFrame.from_dict(sent_dict,orient='columns')

end = time.time()
print(end-start,'s')

0.11033105850219727 s


In [4]:
# トリプルテーブルを作る

from extraction import NERtripleExtraction

triple_dict = {'Message-ID':[],
             'Sentence-ID':[],
             'Triple':[]}

start = time.time()

for i, sentence in enumerate(sentence_df['Sentence']):
    triples = NERtripleExtraction(sentence)
    triple_dict['Message-ID'].extend([sentence_df['Message-ID'][i]] * len(triples))
    triple_dict['Sentence-ID'].extend([sentence_df['Sentence-ID'][i]] * len(triples))
    triple_dict['Triple'].extend(triples)
    
triple_df = pd.DataFrame.from_dict(triple_dict,orient='columns')
        
end = time.time()
print(end-start,'s')

175.74492287635803 s


In [6]:
from extraction import tripleExtraction, entityExtraction, NERtripleExtraction

triple_dict = {'Message-ID':[],
             'Sentence-ID':[],
             'Triple':[]}

start = time.time()

for i, sentence in enumerate(sentence_df['Sentence']):
    triples = tripleExtraction(sentence)
    entities,entities_type = entityExtraction(sentence)
    nertriples = NERtripleExtraction(triples, entities)
    if nertriples is None:
        pass
    else:
        triple_dict['Message-ID'].extend([sentence_df['Message-ID'][i]] * len(nertriples))
        triple_dict['Sentence-ID'].extend([sentence_df['Sentence-ID'][i]] * len(nertriples))
        triple_dict['Triple'].extend(nertriples)

triple_df = pd.DataFrame.from_dict(triple_dict,orient='columns')

end = time.time()
print(end-start,'s')

186.8424050807953 s


In [8]:
output = pd.merge(df[['Path','Message-ID','Subject']],pd.merge(sentence_df,triple_df,on=['Message-ID','Sentence-ID']),on='Message-ID')
output.to_csv('output/allen_p_inbox_triple3.csv')

output = pd.read_csv('output/allen_p_inbox_triple3.csv')

In [7]:
triple_df

,Message-ID,Sentence-ID,Triple
0,<22534132.1075858645229.JavaMail.evans@thyme>,0,"(2001 TVA Valley Authority ( TVA, takes, lead)"
1,<22534132.1075858645229.JavaMail.evans@thyme>,0,"(a division of ... Mexican electricity reform, put, recession delays sector restructuring Dependence on foreign gas)"
2,<22534132.1075858645229.JavaMail.evans@thyme>,0,"(oil increases Utilities slow to buy ASPs, allow, companies to focus Customer service function a key attraction Boosting the capacity of the SPR Calls to boost reserves with stripper production Hesitation attributed to concern over oil price impacts Nabors affiliate to acquire Command Drilling full story ... Georgia Power announces partnership with Nigerian company full story)"
3,<22534132.1075858645229.JavaMail.evans@thyme>,0,"(a key attraction, Boosting, of the SPR Calls)"
4,<22534132.1075858645229.JavaMail.evans@thyme>,0,"(Georgia Power, announces, partnership with Nigerian company)"
...,...,...,...
280,<21572157.1075855377726.JavaMail.evans@thyme>,0,"(You, spin, $ 2,500)"
281,<21572157.1075855377726.JavaMail.evans@thyme>,12,"(to process, take, up to one week)"
282,<7059385.1075855377617.JavaMail.evans@thyme>,7,"(by FreeSoftwarePromotions, Brought, a href=""http://www.freesoftwarepromotions.com/100012"">AOL Members Click Here</a)"
283,<9144651.1075862166126.JavaMail.evans@thyme>,1,"(We, ask, that this confidential information be treated as such , in accordance with applicable laws and regulations governing disclosure of confidential information by gas marketers such as Enron)"
